In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
database = "/Users/duilzhang/Library/CloudStorage/OneDrive-CUHK-Shenzhen/DDA3020/Classification iris.xlsx"
df = pd.read_excel(database)

In [56]:
#Q2.2.1
df.loc[df['class']=='Iris-setosa','name'] = 0
df.loc[df['class']=='Iris-versicolor','name'] = 1
df.loc[df['class']=='Iris-virginica','name'] = 2
X = df[['sepal length','sepal width','petal length','petal width']]
y = df['name']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=26, stratify=y
)
print("Q2.2.1 Split training set and test set:")
print("training set:", X_train.index.tolist())
print("Test set:", X_test.index.tolist())

train_list = ['sepal length','sepal width','petal length','petal width'] 

def iris(c, type, kernel, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test):
    #preparation
    length = 60
    dict = {0:"setosa", 1:"versicolor", 2:"virginica"}
    linear_separate_class = []
    
    #Using svc to train all classes' data, automatically apply the 1vsother method
    
    svm = kernel
    svm.fit(X_train, y_train)
    
    y_train_pred = svm.predict(X_train)
    y_test_pred = svm.predict(X_test)
    
    train_error = 1 - accuracy_score(y_train, y_train_pred)
    test_error = 1 - accuracy_score(y_test, y_test_pred)
    
    print("Total Training Error:",train_error,";","  ","Total Testing Error:",test_error)
    print("-"*length,'\n')
    
    #Using the OneVsRestClassfier to separate one claa from the rest one by one
    ovr = OneVsRestClassifier(svm)
    ovr.fit(X_train, y_train)
    
    for label, coef in enumerate(ovr.estimators_):
        class_train_indices = (y_train == label)
        class_train_error = np.mean(y_train_pred[class_train_indices] != y_train[class_train_indices])
    
        class_test_indices = (y_test == label)
        class_test_error = np.mean(y_test_pred[class_test_indices] != y_test[class_test_indices])
        
        if type == 'linear':
            w = coef.coef_[0]
        else:
            w = []
        b = coef.intercept_[0]
        support_indices = coef.support_
        
        
        print(f"Class {dict[label]}:"
              f"\nTraining Error: {class_train_error:.4f}, Testing Error: {class_test_error:.4f}",
              f"\nWeight vector (w): {w},",f"Bias (b): {b}",
              f"\nSupport vector indices: {support_indices}"
              )
        
        if c == 1e5 and type == 'linear':
            if class_train_error == 0 and class_test_error == 0:
                linear_separate_class.append(dict[label])
            print("-"*length,'\n')
        else:
            support_vectors = X_train.iloc[support_indices]
            support_y = y_train.iloc[support_indices].values
            if c != 1e5:
                prevalue = np.dot(support_vectors,w) + b
                slack = np.maximum(0,1-prevalue*support_y)
                print(f"slack variable:{slack}")
            print("-"*length,'\n')
            
     
    if c == 1e5 and type == 'linear':    
        print(f"Linearly separable classes: {linear_separate_class}")
        print("-"*length,'\n')
    return

Q2.2.1 Split training set and test set:
training set: [41, 8, 142, 146, 104, 16, 33, 57, 105, 148, 77, 50, 80, 31, 120, 81, 121, 129, 9, 144, 126, 39, 91, 135, 68, 97, 42, 20, 49, 76, 115, 44, 37, 12, 18, 145, 73, 133, 108, 11, 61, 102, 22, 82, 112, 15, 70, 29, 40, 45, 93, 75, 54, 113, 131, 7, 53, 118, 64, 130, 89, 69, 36, 65, 127, 84, 2, 122, 35, 67, 47, 117, 134, 52, 101, 10, 71, 79, 124, 92, 78, 136, 86, 88, 100, 0, 83, 66, 140, 107, 43, 3, 114, 58, 28, 55, 14, 106, 5, 27, 46, 128, 103, 99, 23]
Test set: [48, 32, 90, 96, 51, 26, 125, 147, 87, 110, 34, 1, 38, 137, 6, 24, 25, 149, 63, 62, 138, 94, 60, 59, 111, 109, 116, 30, 21, 13, 98, 139, 95, 4, 17, 19, 132, 119, 72, 85, 141, 143, 123, 74, 56]


In [57]:
#Q2.2.2 
C1 = 1e5
kernel = SVC(kernel='linear', C=C1)
print("Q2.2.2 Calculation using Standard SVM Model:")
iris(C1,type='linear',kernel=kernel)

Q2.2.2 Calculation using Standard SVM Model:
Total Training Error: 0.01904761904761909 ;    Total Testing Error: 0.022222222222222254
------------------------------------------------------------ 

Class setosa:
Training Error: 0.0000, Testing Error: 0.0000 
Weight vector (w): [ 0.05136527  0.39964415 -0.92633526 -0.29691361], Bias (b): 1.1428018956287134 
Support vector indices: [  7   0 104]
------------------------------------------------------------ 

Class versicolor:
Training Error: 0.0286, Testing Error: 0.0667 
Weight vector (w): [ 0.72327882 -3.76613166  1.280073   -4.06767525], Bias (b): 6.23875152474661 
Support vector indices: [  0   1   2   3   4  17  18  19  20  23  26  27  28  32  37  38  57  59
  66  67  72  74  89  97  99 102   7  10  11  12  15  22  25  29  36  40
  46  50  51  52  58  60  63  65  73  80  82  83  93  95 103]
------------------------------------------------------------ 

Class virginica:
Training Error: 0.0286, Testing Error: 0.0000 
Weight vector (w): 

In [58]:
#Q2.2.3
print("Calculation using SVM with Slack Variables (C=0.25×t, where t=1,...,4):")
for t in [1,2,3,4]:
    C2 = 0.25*t
    kernel = SVC(kernel='linear', C=C2)
    print(f"C = {C2}:")
    iris(C2,type='linear',kernel=kernel)

Calculation using SVM with Slack Variables (C=0.25×t, where t=1,...,4):
C = 0.25:
Total Training Error: 0.04761904761904767 ;    Total Testing Error: 0.022222222222222254
------------------------------------------------------------ 

Class setosa:
Training Error: 0.0000, Testing Error: 0.0000 
Weight vector (w): [-0.02286395  0.37077471 -0.85358045 -0.29416891], Bias (b): 1.4477182910672204 
Support vector indices: [  7  50  77   0  31 104]
slack variable:[1.88544014 1.92480401 2.00029245 1.         1.         1.        ]
------------------------------------------------------------ 

Class versicolor:
Training Error: 0.0286, Testing Error: 0.0000 
Weight vector (w): [ 0.09090571 -0.7343008   0.14083506 -0.43461725], Bias (b): 0.8534882851080848 
Support vector indices: [  0   1   2   3   4   8  16  17  18  20  23  26  27  32  33  37  38  41
  44  47  49  53  57  59  64  66  67  68  70  72  74  89  91  92  97 101
 102   7  10  11  12  15  22  24  25  29  36  40  43  46  50  51  52  56
 

In [59]:
C3 = 1e5
poly2 = SVC(kernel='poly', degree=2, C=C3, coef0=1)
poly3 = SVC(kernel='poly', degree=3, C=C3, coef0=1)
rrf = SVC(kernel='rbf',gamma=1)
sig = SVC(kernel='sigmoid',gamma=1)
k = {"(a) 2nd-order Polynomial Kernel:":poly2,
     "(b) 3rd-order Polynomial Kernel:":poly3,
     "(c) Radial Basis Function Kernel with σ = 1:":rrf,
     "(d) Sigmoidal Kernel with σ = 1:":sig}
for i in k:
    print('\n')
    print(i)
    iris(C3,'nonlinear',k[i])



(a) 2nd-order Polynomial Kernel:
Total Training Error: 0.0 ;    Total Testing Error: 0.0444444444444444
------------------------------------------------------------ 

Class setosa:
Training Error: 0.0000, Testing Error: 0.0000 
Weight vector (w): [], Bias (b): 1.096893684517965 
Support vector indices: [  7   0 104]
------------------------------------------------------------ 

Class versicolor:
Training Error: 0.0000, Testing Error: 0.0667 
Weight vector (w): [], Bias (b): -3.714270928763049 
Support vector indices: [37 45 64 72 96 97  7 10 36 46 58 86]
------------------------------------------------------------ 

Class virginica:
Training Error: 0.0000, Testing Error: 0.0667 
Weight vector (w): [], Bias (b): 93.54377281944193 
Support vector indices: [ 7 10 46 86  3 37 38 64 72 97]
------------------------------------------------------------ 



(b) 3rd-order Polynomial Kernel:
Total Training Error: 0.0 ;    Total Testing Error: 0.06666666666666665
--------------------------------